<a href="https://colab.research.google.com/github/okwasna/dane_wszystkie/blob/main/dane_wszystkie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas


In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score


In [4]:

# Wczytanie danych z pliku Excel
file_path = 'summary_htseq_norm2.xlsx'
df = pd.read_excel(file_path, sheet_name='summary_htseq')
# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")

# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")

# Obliczenie średniej i wariancji kolumny 'var' przed filtrowaniem
mean_before_filtering = df['var'].mean()
variance_before_filtering = df['var'].var()

print(f"Średnia wariancji przed filtrowaniem: {mean_before_filtering}")
print(f"Wariancja przed filtrowaniem: {variance_before_filtering}")

# Krok 1: Filtrowanie genów o niskiej wariancji
# Ustalmy próg wariancji na 10 - odrzucamy geny poniżej tej wartości
variance_threshold = 10

# Filtrowanie genów o wariancji poniżej progu
df_filtered = df[df['var'] >= variance_threshold]

# Obliczenie średniej i wariancji kolumny 'var' po filtrowaniu
mean_after_filtering = df_filtered['var'].mean()
variance_after_filtering = df_filtered['var'].var()

print(f"Średnia wariancji po filtrowaniu: {mean_after_filtering}")
print(f"Wariancja po filtrowaniu: {variance_after_filtering}")

# Zliczenie ilości genów dla lncRNA przed filtrowaniem
initial_counts = df['lncRNA'].value_counts()

# Zliczenie ilości genów dla lncRNA po filtrowaniu
filtered_counts = df_filtered['lncRNA'].value_counts()

# Obliczenie liczby usuniętych genów z lncRNA 0 i 1
removed_counts = initial_counts - filtered_counts

# Wyświetlenie liczby usuniętych genów osobno dla lncRNA = 0 i lncRNA = 1
removed_0 = removed_counts.get(0, 0)
removed_1 = removed_counts.get(1, 0)

print(f"Liczba genów usuniętych z lncRNA = 0: {removed_0}")
print(f"Liczba genów usuniętych z lncRNA = 1: {removed_1}")

# Wyświetlenie łącznej liczby genów po filtrowaniu
total_genes_after_filtering = len(df_filtered)
print(f"Łączna liczba genów po filtrowaniu: {total_genes_after_filtering}")

Liczba genów przed filtrowaniem: 28677
Liczba genów przed filtrowaniem: 28677
Średnia wariancji przed filtrowaniem: 83837021.10925484
Wariancja przed filtrowaniem: 1.4257388777828288e+20
Średnia wariancji po filtrowaniu: 108068243.92073244
Wariancja po filtrowaniu: 1.837809291273408e+20
Liczba genów usuniętych z lncRNA = 0: 3473
Liczba genów usuniętych z lncRNA = 1: 2957
Łączna liczba genów po filtrowaniu: 22247


In [ ]:
# Wybór kolumn zaczynających się od SRR7451446
data_values = df_filtered.loc[:, 'SRR7451446':]  # Wybieramy wszystkie kolumny od SRR7451446
# Lista do przechowywania wartości wskaźnika silhouette dla każdej liczby klastrów
silhouette_scores = []



# Klasteryzacja k-means dla liczby klastrów od 45 do 55
for n_clusters in range(45, 56):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(scaled_data_values)

    # Dodanie informacji o klastrze do oryginalnych danych
    df_filtered[f'Cluster_{n_clusters}'] = clusters

    # Obliczenie wskaźnika silhouette dla każdego punktu
    silhouette_vals = silhouette_samples(scaled_data_values, clusters)

    # Dodanie wskaźnika silhouette do danych
    df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals

    # Liczba genów w każdym klastrze
    cluster_counts = df_filtered[f'Cluster_{n_clusters}'].value_counts()

    # Średnia wartość wskaźnika silhouette dla każdego klastra
    silhouette_per_cluster = df_filtered.groupby(f'Cluster_{n_clusters}')[f'Silhouette_{n_clusters}'].mean()

    # Wyświetlenie razem liczby genów i średniego wskaźnika silhouette dla każdego klastra
    print(f"\nPodsumowanie dla {n_clusters} klastrów:")
    for cluster_num in cluster_counts.index:
        count = cluster_counts[cluster_num]
        silhouette_avg = silhouette_per_cluster[cluster_num]
        # Wyświetlenie w oddzielnych liniach
        print(f"Klaster {cluster_num}:")
        print(f" Liczba genów = {count},")
        print(f" Średnia wartość silhouette = {silhouette_avg:.6f}")


<ipython-input-43-d6464eeeb07c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Cluster_{n_clusters}'] = clusters
<ipython-input-43-d6464eeeb07c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals



Podsumowanie dla 45 klastrów:
Klaster 17:
 Liczba genów = 1407,
 Średnia wartość silhouette = 0.166327
Klaster 35:
 Liczba genów = 1029,
 Średnia wartość silhouette = 0.122669
Klaster 3:
 Liczba genów = 847,
 Średnia wartość silhouette = 0.084875
Klaster 23:
 Liczba genów = 798,
 Średnia wartość silhouette = 0.073575
Klaster 32:
 Liczba genów = 793,
 Średnia wartość silhouette = 0.092071
Klaster 11:
 Liczba genów = 703,
 Średnia wartość silhouette = 0.093137
Klaster 26:
 Liczba genów = 694,
 Średnia wartość silhouette = 0.112277
Klaster 38:
 Liczba genów = 656,
 Średnia wartość silhouette = 0.121037
Klaster 21:
 Liczba genów = 652,
 Średnia wartość silhouette = 0.078989
Klaster 39:
 Liczba genów = 646,
 Średnia wartość silhouette = 0.139100
Klaster 4:
 Liczba genów = 644,
 Średnia wartość silhouette = 0.086052
Klaster 22:
 Liczba genów = 569,
 Średnia wartość silhouette = 0.019997
Klaster 41:
 Liczba genów = 560,
 Średnia wartość silhouette = 0.086975
Klaster 1:
 Liczba genów = 549,
 

<ipython-input-43-d6464eeeb07c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Cluster_{n_clusters}'] = clusters
<ipython-input-43-d6464eeeb07c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals



Podsumowanie dla 46 klastrów:
Klaster 17:
 Liczba genów = 1337,
 Średnia wartość silhouette = 0.166045
Klaster 35:
 Liczba genów = 1014,
 Średnia wartość silhouette = 0.124044
Klaster 3:
 Liczba genów = 865,
 Średnia wartość silhouette = 0.081940
Klaster 23:
 Liczba genów = 790,
 Średnia wartość silhouette = 0.078176
Klaster 32:
 Liczba genów = 760,
 Średnia wartość silhouette = 0.081942
Klaster 1:
 Liczba genów = 701,
 Średnia wartość silhouette = 0.055874
Klaster 39:
 Liczba genów = 683,
 Średnia wartość silhouette = 0.140028
Klaster 26:
 Liczba genów = 663,
 Średnia wartość silhouette = 0.119142
Klaster 21:
 Liczba genów = 645,
 Średnia wartość silhouette = 0.082903
Klaster 38:
 Liczba genów = 640,
 Średnia wartość silhouette = 0.124261
Klaster 4:
 Liczba genów = 633,
 Średnia wartość silhouette = 0.119327
Klaster 11:
 Liczba genów = 603,
 Średnia wartość silhouette = 0.072009
Klaster 41:
 Liczba genów = 562,
 Średnia wartość silhouette = 0.066403
Klaster 22:
 Liczba genów = 552,
 

<ipython-input-43-d6464eeeb07c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Cluster_{n_clusters}'] = clusters
<ipython-input-43-d6464eeeb07c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals


...

<ipython-input-43-d6464eeeb07c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Cluster_{n_clusters}'] = clusters
<ipython-input-43-d6464eeeb07c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals



Podsumowanie dla 54 klastrów:
Klaster 17:
 Liczba genów = 1144,
 Średnia wartość silhouette = 0.145789
Klaster 3:
 Liczba genów = 817,
 Średnia wartość silhouette = 0.086162
Klaster 1:
 Liczba genów = 804,
 Średnia wartość silhouette = 0.092350
Klaster 23:
 Liczba genów = 722,
 Średnia wartość silhouette = 0.080613
Klaster 35:
 Liczba genów = 722,
 Średnia wartość silhouette = 0.090335
Klaster 21:
 Liczba genów = 636,
 Średnia wartość silhouette = 0.108440
Klaster 39:
 Liczba genów = 632,
 Średnia wartość silhouette = 0.123640
Klaster 4:
 Liczba genów = 621,
 Średnia wartość silhouette = 0.107856
Klaster 32:
 Liczba genów = 609,
 Średnia wartość silhouette = 0.062616
Klaster 26:
 Liczba genów = 592,
 Średnia wartość silhouette = 0.126268
Klaster 11:
 Liczba genów = 590,
 Średnia wartość silhouette = 0.072258
Klaster 51:
 Liczba genów = 522,
 Średnia wartość silhouette = 0.029219
Klaster 41:
 Liczba genów = 516,
 Średnia wartość silhouette = 0.068625
Klaster 38:
 Liczba genów = 509,
 Ś

<ipython-input-43-d6464eeeb07c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Cluster_{n_clusters}'] = clusters



Podsumowanie dla 55 klastrów:
Klaster 35:
 Liczba genów = 1121,
 Średnia wartość silhouette = 0.145339
Klaster 18:
 Liczba genów = 817,
 Średnia wartość silhouette = 0.066101
Klaster 26:
 Liczba genów = 789,
 Średnia wartość silhouette = 0.085550
Klaster 42:
 Liczba genów = 693,
 Średnia wartość silhouette = 0.090619
Klaster 34:
 Liczba genów = 689,
 Średnia wartość silhouette = 0.101444
Klaster 1:
 Liczba genów = 668,
 Średnia wartość silhouette = 0.084191
Klaster 11:
 Liczba genów = 630,
 Średnia wartość silhouette = 0.127101
Klaster 10:
 Liczba genów = 606,
 Średnia wartość silhouette = 0.066946
Klaster 14:
 Liczba genów = 589,
 Średnia wartość silhouette = 0.047359
Klaster 41:
 Liczba genów = 536,
 Średnia wartość silhouette = 0.081959
Klaster 13:
 Liczba genów = 526,
 Średnia wartość silhouette = 0.065165
Klaster 8:
 Liczba genów = 522,
 Średnia wartość silhouette = 0.112297
Klaster 39:
 Liczba genów = 521,
 Średnia wartość silhouette = 0.095027
Klaster 48:
 Liczba genów = 502,
 

<ipython-input-43-d6464eeeb07c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[f'Silhouette_{n_clusters}'] = silhouette_vals


In [ ]:
#zrobić wizualizacje (heatmapa + wykres)

In [ ]:
# Klasteryzacja DBSCAN z większym eps i mniejszym min_samples
dbscan = DBSCAN(eps=3.0, min_samples=5, metric='euclidean')  # Zwiększamy wartość eps do 3.0 i zmniejszamy min_samples
dbscan_clusters = dbscan.fit_predict(scaled_data_values)

# Dodanie wyników DBSCAN do oryginalnych danych
df_filtered['DBSCAN_Cluster'] = dbscan_clusters

# Wyświetlenie liczby genów w każdym klastrze DBSCAN
print(f'\nLiczba genów w każdym klastrze DBSCAN:')
print(df_filtered['DBSCAN_Cluster'].value_counts())


Liczba genów w każdym klastrze DBSCAN:
DBSCAN_Cluster
 0    17782
-1     4450
 1        7
 3        5
 2        3
Name: count, dtype: int64


<ipython-input-38-14f9611c286a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['DBSCAN_Cluster'] = dbscan_clusters


In [ ]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering


# Wybór kolumn z wartościami eksperymentalnymi (od SRR7451446 do SRR7451472)
data_values = df.loc[:, 'SRR7451446':'SRR7451472']

# 1. Obliczenie macierzy odległości opartej na korelacji Pearsona
correlation_matrix = 1 - pdist(data_values, metric='correlation')

# Normalizacja wartości na przedział [0, 1]
correlation_matrix = (correlation_matrix - correlation_matrix.min()) / (correlation_matrix.max() - correlation_matrix.min())

# 2. Klasteryzacja hierarchiczna z wybraną liczbą klastrów (np. 5 klastrów)
agg_clustering = AgglomerativeClustering(n_clusters=5, metric='precomputed', linkage='average')
clusters = agg_clustering.fit_predict(squareform(correlation_matrix))

# 3. Dodanie wyników klasteryzacji do oryginalnych danych
df['Hierarchical_Cluster'] = clusters

# 4. Wyświetlenie liczby genów w każdym klastrze
print(f'\nLiczba genów w każdym klastrze Hierarchicznej Klasteryzacji:')
print(df['Hierarchical_Cluster'].value_counts())


NameError: name 'df' is not defined

poprawic dbscan, clustering, zrobic na danych bez usuwania wariancji co wyjdzie
